In [2]:
#Text Embedding for Question and Answering V02


In [3]:
#This project serves as a proof of concept for incorporating a text file with OpenAI 

import os
import openai
import tiktoken


openai.api_key = os.getenv("OPENAI_API_KEY")

#ask about a company that is not in the Davinci Training set training set
#To help us try and deal with hallucination, we are going to use formatting?

prompt = '''Only answer the question below if you have 100% certainty of the facts.
Q:  What does the start up company Pentera do and who invested in it?

A: '''


response = openai.Completion.create(prompt = prompt, 
                                    temperature = 0, 
                                    max_tokens=512, 
                                    model = 'text-davinci-003')

print(response['choices'][0]['text'])

 I cannot answer this question with 100% certainty.


In [4]:
import pandas as pd
df = pd.read_csv("unicorns.csv")

df.head()

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website
0,"10/31/2022, 2:37:05 AM",Esusu,https://www.cbinsights.com/company/esusu,1.0,1/27/2022,2022,New York,United States,Fintech,"[""Next Play Ventures"",""Zeal Capital Partners"",...",NaN
1,"10/31/2022, 2:37:05 AM",Fever Labs,https://www.cbinsights.com/company/fever-labs,1.0,1/26/2022,2022,New York,United States,Internet software & services,"[""Accel"",""14W"",""GS Growth""]",NaN
2,"10/31/2022, 2:37:04 AM",Minio,https://www.cbinsights.com/company/minio,1.0,1/26/2022,2022,Palo Alto,United States,Data management & analytics,"[""General Catalyst"",""Nexus Venture Partners"",""...",NaN
3,"10/31/2022, 2:37:04 AM",Darwinbox,https://www.cbinsights.com/company/darwinbox,1.0,1/25/2022,2022,Hyderabad,India,Internet software & services,"[""Lightspeed India Partners"",""Sequoia Capital ...",NaN
4,"10/31/2022, 2:37:04 AM",Pentera,https://www.cbinsights.com/company/pcysys,1.0,1/11/2022,2022,Petah Tikva,Israel,Cybersecurity,"[""AWZ Ventures"",""Blackstone"",""Insight Partners""]",NaN


In [5]:
import ast

def summary(company, crunchbase_url, city, country, industry, investor_list):
    investors = "The investors in the company are"
    
    for investor in ast.literal_eval(investor_list):
        investors += f" {investor},"
        
    text = f"{company} has headquarters in {city} in {country} and is in the field of {industry}. {investors}.  More info at {crunchbase_url}"
    
    return text

In [6]:
df['summary'] = df.apply(lambda df: summary(df['Company'], df['Crunchbase Url'],df['City'], df['Country'], df['Industry'], df['Investors']), axis =1)

In [7]:
df['summary']

0       Esusu has headquarters in New York in United S...
1       Fever Labs has headquarters in New York in Uni...
2       Minio has headquarters in Palo Alto in United ...
3       Darwinbox has headquarters in Hyderabad in Ind...
4       Pentera has headquarters in Petah Tikva in Isr...
                              ...                        
1194    Fanatics has headquarters in Jacksonville in U...
1195    SpaceX has headquarters in Hawthorne in United...
1196    Vice Media has headquarters in Brooklyn in Uni...
1197    Klarna has headquarters in Stockholm in Sweden...
1198    Veepee has headquarters in La Plaine Saint-Den...
Name: summary, Length: 1199, dtype: object

In [8]:
#Token Count and Pricing
#!pip install tiktoken

import tiktoken

def num_tokens_from_string(string, encoding_name):
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(df['summary'][0],"cl100k_base")

53

In [9]:
df['token_count'] = df['summary'].apply(lambda text: num_tokens_from_string(text,"cl100k_base"))
df

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website,summary,token_count
0,"10/31/2022, 2:37:05 AM",Esusu,https://www.cbinsights.com/company/esusu,1.00,1/27/2022,2022,New York,United States,Fintech,"[""Next Play Ventures"",""Zeal Capital Partners"",...",NaN,Esusu has headquarters in New York in United S...,53
1,"10/31/2022, 2:37:05 AM",Fever Labs,https://www.cbinsights.com/company/fever-labs,1.00,1/26/2022,2022,New York,United States,Internet software & services,"[""Accel"",""14W"",""GS Growth""]",NaN,Fever Labs has headquarters in New York in Uni...,55
2,"10/31/2022, 2:37:04 AM",Minio,https://www.cbinsights.com/company/minio,1.00,1/26/2022,2022,Palo Alto,United States,Data management & analytics,"[""General Catalyst"",""Nexus Venture Partners"",""...",NaN,Minio has headquarters in Palo Alto in United ...,52
3,"10/31/2022, 2:37:04 AM",Darwinbox,https://www.cbinsights.com/company/darwinbox,1.00,1/25/2022,2022,Hyderabad,India,Internet software & services,"[""Lightspeed India Partners"",""Sequoia Capital ...",NaN,Darwinbox has headquarters in Hyderabad in Ind...,57
4,"10/31/2022, 2:37:04 AM",Pentera,https://www.cbinsights.com/company/pcysys,1.00,1/11/2022,2022,Petah Tikva,Israel,Cybersecurity,"[""AWZ Ventures"",""Blackstone"",""Insight Partners""]",NaN,Pentera has headquarters in Petah Tikva in Isr...,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,"10/31/2022, 2:32:42 AM",Fanatics,https://www.cbinsights.com/company/fanatics,27.00,6/6/2012,2012,Jacksonville,United States,E-commerce & direct-to-consumer,"[""SoftBank Group"",""Andreessen Horowitz"",""Temas...",NaN,Fanatics has headquarters in Jacksonville in U...,58
1195,"10/31/2022, 2:32:38 AM",SpaceX,https://www.cbinsights.com/company/space-explo...,127.00,12/1/2012,2012,Hawthorne,United States,Other,"[""Founders Fund"",""Draper Fisher Jurvetson"",""Ro...",NaN,SpaceX has headquarters in Hawthorne in United...,60
1196,"10/31/2022, 2:33:10 AM",Vice Media,https://www.cbinsights.com/company/vice-media,5.70,4/2/2011,2011,Brooklyn,United States,Internet software & services,"[""Technology Crossover Ventures"",""A&E Televisi...",NaN,Vice Media has headquarters in Brooklyn in Uni...,52
1197,"10/31/2022, 2:33:05 AM",Klarna,https://www.cbinsights.com/company/klarna,6.70,12/12/2011,2011,Stockholm,Sweden,Fintech,"[""Institutional Venture Partners"",""Sequoia Cap...",NaN,Klarna has headquarters in Stockholm in Sweden...,52


In [10]:
df['token_count'].sum()

64650

In [11]:
df['token_count'].sum()*0.0004/1000

0.025860000000000005

In [12]:
df['summary'][0]

'Esusu has headquarters in New York in United States and is in the field of Fintech. The investors in the company are Next Play Ventures, Zeal Capital Partners, SoftBank Group,.  More info at https://www.cbinsights.com/company/esusu'

In [13]:
def get_embedding(text):
    result = openai.Embedding.create(
        model = 'text-embedding-ada-002', input=text)
    return result['data'][0]['embedding']

get_embedding('Esusu has headquarters in New York in United States and is in the field of Fintech. The investors in the company are Next Play Ventures, Zeal Capital Partners, SoftBank Group,.  More info at https://www.cbinsights.com/company/esusu')

[0.011399493552744389,
 -0.01880882866680622,
 -0.023927517235279083,
 -0.03205561637878418,
 -0.01481866929680109,
 0.008799844421446323,
 -0.019803009927272797,
 0.02418277971446514,
 0.004100996069610119,
 -0.022516854107379913,
 0.01941339671611786,
 -0.014106621034443378,
 0.004537629894912243,
 -0.017908690497279167,
 0.009565632790327072,
 -0.024411171674728394,
 -0.0019598128274083138,
 -0.021482368931174278,
 0.017586253583431244,
 0.00214790110476315,
 -0.02351103536784649,
 -0.013280375860631466,
 -0.012044367380440235,
 0.02161671780049801,
 -0.019561180844902992,
 0.002942238003015518,
 -0.014187230728566647,
 -0.0010680727427825332,
 -0.009310370311141014,
 -0.010781489312648773,
 0.011661473661661148,
 -0.0012141769984737039,
 -0.019628355279564857,
 -0.0007418571040034294,
 -0.014899278990924358,
 0.004944034852087498,
 -0.022798987105488777,
 0.015933765098452568,
 0.02020605467259884,
 -0.01076805405318737,
 0.031679440289735794,
 0.00875953957438469,
 -0.015181411057

In [18]:
vector = get_embedding('Esusu has headquarters in New York in United States and is in the field of Fintech. The investors in the company are Next Play Ventures, Zeal Capital Partners, SoftBank Group,.  More info at https://www.cbinsights.com/company/esusu')

In [19]:
len(vector)

1536

In [20]:
df['embedding'] = df['summary'].apply(get_embedding)

In [21]:
df.to_csv('my_embedding.csv')

In [22]:
df['embedding']

0       [0.011399493552744389, -0.01880882866680622, -...
1       [0.00775443110615015, 0.011641528457403183, -0...
2       [0.004580601118505001, -0.03932548314332962, 0...
3       [-0.0004160037206020206, -0.025348013266921043...
4       [0.012735920958220959, -0.010580189526081085, ...
                              ...                        
1194    [-0.001020857598632574, -0.009246503002941608,...
1195    [0.02621828019618988, -0.013919810764491558, -...
1196    [0.006367433350533247, -0.029810570180416107, ...
1197    [0.024655524641275406, -0.023734185844659805, ...
1198    [0.002615401055663824, -0.01281877513974905, 0...
Name: embedding, Length: 1199, dtype: object

In [24]:
#We now have all our summary text and their vector embedding.

#Now what we'll need to do is the following:  
#Embed a query string to vector
#Perform a cosine similarity between query vector and all our document vectors

#Choose most similar and inject context

prompt = "What does the company Pentera do and who invested in it?"

prompt_embedding = get_embedding(prompt)

import numpy as np
def vector_similarity(vec1, vec2):
    return np.dot(np.array(vec1), np.array(vec2))

df['prompt_similarity'] = df['embedding'].apply(lambda vector:vector_similarity(vector, prompt_embedding))

df['prompt_similarity']

df.nlargest(1, 'prompt_similarity')

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website,summary,token_count,embedding,prompt_similarity
4,"10/31/2022, 2:37:04 AM",Pentera,https://www.cbinsights.com/company/pcysys,1.0,1/11/2022,2022,Petah Tikva,Israel,Cybersecurity,"[""AWZ Ventures"",""Blackstone"",""Insight Partners""]",NaN,Pentera has headquarters in Petah Tikva in Isr...,53,"[0.012735920958220959, -0.010580189526081085, ...",0.881048


In [25]:
#Note:  We should be looking at other programs such as Pinecone for vector similarity search for LLMs.



In [26]:
df.nlargest(1, 'prompt_similarity').iloc[0]['summary']

'Pentera has headquarters in Petah Tikva in Israel and is in the field of Cybersecurity . The investors in the company are AWZ Ventures, Blackstone, Insight Partners,.  More info at https://www.cbinsights.com/company/pcysys'

In [27]:
context = df.nlargest(1, 'prompt_similarity').iloc[0]['summary']

In [28]:
prompt = f'''Only answer the question below if you have 100% certainty of the facts.
Context: {context}
Q:  What does the start up company Pentera do and who invested in it?

A:'''

In [29]:
print(prompt)

Only answer the question below if you have 100% certainty of the facts.
Context: Pentera has headquarters in Petah Tikva in Israel and is in the field of Cybersecurity . The investors in the company are AWZ Ventures, Blackstone, Insight Partners,.  More info at https://www.cbinsights.com/company/pcysys
Q:  What does the start up company Pentera do and who invested in it?

A:


In [30]:
response = openai.Completion.create(prompt = prompt, 
                                    temperature = 0, 
                                    max_tokens=512, 
                                    model = 'text-davinci-003')
print(response['choices'][0]['text'])

 Pentera is a Cybersecurity start up company and it has been invested in by AWZ Ventures, Blackstone, and Insight Partners.


In [ ]:
def embed_prompt_lookup():
    question = input ("What question do you have about a Unicorn start-up?")
    
    prompt_embedding = get_embedding(question)
    
    df['prompt_similarity'] = df['embedding'].apply(lambda vector:vector_similarity(vector, prompt_embedding))
    
    context = df.nlargest(1, 'prompt_similarity').iloc[0]['summary']
    
    prompt = f'''Only answer the question below if you have 100% certainty of the facts.
                    Context: {context}
                    Q: {question}?
                    A:'''
    response = openai.Completion.create(
        prompt = prompt, 
        temperature = 0, 
        max_tokens=512, 
        model = 'text-davinci-003')
    
    print(response['choices'][0]['text'])
    
embed_prompt_lookup()